## Query 10

Quiero buscar los pares de productos que son frecuentemente comprados juntos. Para esto buscamos en las órdenes aquellos productos que aparecen juntos en la misma orden. Luego, contamos cuántas veces aparece cada par de productos juntos y devolvemos los pares más frecuentes. Cabe aclarar que si se compran varias unidades de AMBOS productos en la misma orden, se cuenta como un par.

Para hacer esto debemos seguir los siguientes pasos:

1. Importar el dataset de order_items (data/raw/order_items.csv).
2. Filtrar por valores inválidos (order_id, product_id) y convertir product_id a string.
3. Mapear para obtener (order_id, product_id).
4. Agrupar por order_id para obtener lista de productos por orden: (order_id, [product_id, ...]).
5. Para cada orden, generar todos los pares posibles de productos (combinaciones sin repetición).
6. Mapear a ((product_id_1, product_id_2), 1) donde product_id_1 < product_id_2 para evitar duplicados, contando 1 por cada orden.
7. Reducir por par de productos sumando las veces que aparecen juntos.
8. Tomar los top 10 pares más frecuentes.
9. Broadcastear los IDs únicos de productos de los pares más frecuentes (convertidos a string).
10. Importar el dataset de products (data/raw/products.csv).
11. Filtrar productos con datos válidos (product_id, product_name), convertir product_id a string y filtrar solo los que están en el broadcast.
12. Paralelizar los top 10 pares para crear RDD.
13. Hacer primer join con products_kv usando product_id_1 como clave para obtener el nombre del primer producto.
14. Hacer segundo join con products_kv usando product_id_2 como clave para obtener el nombre del segundo producto.
15. Formatear resultado final extrayendo solo los nombres de productos y el conteo.
16. Mostrar los resultados en formato de tabla con pandas.

In [59]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ParesProductos").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [60]:
items_df = spark.read.csv(
    "../../data/raw/order_items.csv", header=True, inferSchema=True
)
items_rdd = items_df.rdd

items_by_order = (
    items_rdd.filter(lambda r: r.order_id and r.product_id)
    .map(lambda r: (r.order_id, str(r.product_id)))
    .groupByKey()
    .mapValues(list)
)  # (order_id, [product_id, ...])

In [61]:
from itertools import combinations


def generate_pairs(order_items):
    """Generate all product pairs from an order, counting 1 per order"""
    pairs = []
    items_list = list(order_items)

    # Generate all combinations of 2 products
    for prod1, prod2 in combinations(items_list, 2):
        if prod1 < prod2:
            pair = ((prod1, prod2), 1)
        else:
            pair = ((prod2, prod1), 1)
        pairs.append(pair)

    return pairs


product_pairs = items_by_order.flatMap(
    lambda kv: generate_pairs(kv[1])
)  # ((product_id_1, product_id_2), 1)

pair_counts = product_pairs.reduceByKey(
    lambda a, b: a + b
)  # ((product_id_1, product_id_2), veces_juntos)

top10 = pair_counts.takeOrdered(10, key=lambda kv: -kv[1])

In [62]:
ids_top = spark.sparkContext.broadcast(
    {str(p) for (p1, p2), _ in top10 for p in (p1, p2)}
)

products_df = spark.read.csv(
    "../../data/raw/products.csv", header=True, inferSchema=True
)

products_rdd = products_df.rdd

products_kv = (
    products_rdd.filter(lambda r: r.product_id is not None and r.product_name)
    .map(lambda r: (str(r.product_id), r.product_name))
    .filter(lambda kv: kv[0] in ids_top.value)
)  # RDD[(product_id, product_name)]

In [63]:
top10_rdd = spark.sparkContext.parallelize(top10)  # [((p1, p2), cnt)]

pairs_by_p1 = top10_rdd.map(lambda kv: (kv[0][0], (kv[0][1], kv[1])))
j1 = pairs_by_p1.join(products_kv)  # (p1, ((p2, cnt), name1))

to_p2 = j1.map(lambda kv: (kv[1][0][0], (kv[1][0][1], kv[1][1])))
j2 = to_p2.join(products_kv)  # (p2, ((cnt, name1), name2))

with_names = j2.map(lambda kv: ((kv[1][0][1], kv[1][1]), kv[1][0][0]))
result = with_names.collect()  # ((name1, name2), cnt)

In [64]:
import pandas as pd

if result:
    results_formatted = [
        {
            "producto_1": name1,
            "producto_2": name2,
            "veces_comprados_juntos": cnt,
        }
        for (name1, name2), cnt in result
    ]
    df = pd.DataFrame(results_formatted)
    display(df)
else:
    print("No results")

,producto_1,producto_2,veces_comprados_juntos
0,Progressive methodical open architecture,Universal solution-oriented hub,2
1,front-line mobile core,User-centric mobile task-force,2
2,Total reciprocal core,Customer-focused fresh-thinking open architecture,2
3,Profound zero tolerance forecast,User-centric next generation pricing structure,2
4,Stand-alone static implementation,Right-sized regional approach,2
5,Business-focused didactic adapter,Monitored contextually-based Local Area Network,2
6,Switchable neutral forecast,Exclusive empowering strategy,2
7,USER-CENTRIC 6THGENERATION INTERNET SOLUTION,Integrated web-enabled challenge,2
8,Re-engineered regional help-desk,Automated executive leverage,2
